In [57]:
# Import minimal requirements
import requests
import json
import re

In [58]:
# Set the base URL for the ARAX reasoner and its endpoint
endpoint_url = 'https://arax.rtx.ai/devED/api/rtx/v1/query'

# Create a dict of the request, specifying the list of DSL commands
request = { "previous_message_processing_plan": { "processing_actions": [
            "add_qnode(name=DOID:14330, id=n00)",
            "add_qnode(type=protein, is_set=true, id=n01)",
            "add_qnode(type=chemical_substance, id=n02)",
            "add_qedge(source_id=n00, target_id=n01, id=e00)",
            "add_qedge(source_id=n01, target_id=n02, id=e01, type=physically_interacts_with)",
            "expand(edge_id=[e00,e01])",
            "overlay(action=compute_jaccard, start_node_id=n00, intermediate_node_id=n01, end_node_id=n02, virtual_edge_type=J1)",
            "filter_kg(action=remove_edges_by_attribute, edge_attribute=jaccard_index, direction=below, threshold=.2, remove_connected_nodes=t, qnode_id=n02)",
            "filter_kg(action=remove_edges_by_property, edge_property=provided_by, property_value=Pharos)",
            "resultify(ignore_edge_direction=true)",
            "return(message=true, store=true)",
            ] } }

In [59]:
# Send the request to RTX and check the status
print(f"Executing query at {endpoint_url}\nPlease wait...")
response_content = requests.post(endpoint_url, json=request, headers={'accept': 'application/json'})
status_code = response_content.status_code
if status_code != 200:
    print("ERROR returned with status "+str(status_code))
    print(response_content.json())
else:
    print(f"Response returned with status {status_code}")

Executing query at https://arax.rtx.ai/devED/api/rtx/v1/query
Please wait...
Response returned with status 200


In [60]:
# Unpack the response content into a dict
response_dict = response_content.json()
print(json.dumps(response_dict, indent=2, sort_keys=True))

{
  "code_description": "Normal completion",
  "context": "https://raw.githubusercontent.com/biolink/biolink-model/master/context.jsonld",
  "datetime": "2020-03-03 17:44:59",
  "id": "https://arax.rtx.ai/api/rtx/v1/message/1355",
  "knowledge_graph": {
    "edges": [
      {
        "id": "1346604",
        "is_defined_by": "RTX",
        "provided_by": "BioLink",
        "qedge_id": "e00",
        "relation": "associated_with_disease",
        "source_id": "UniProtKB:P02675",
        "target_id": "DOID:14330",
        "type": "gene_associated_with_condition"
      },
      {
        "id": "1357727",
        "is_defined_by": "RTX",
        "provided_by": "DisGeNet",
        "qedge_id": "e00",
        "relation": "gene_associated_with_condition",
        "source_id": "UniProtKB:P01375",
        "target_id": "DOID:14330",
        "type": "gene_associated_with_condition"
      },
      {
        "id": "1338488",
        "is_defined_by": "RTX",
        "provided_by": "BioLink",
        "q

In [54]:
# Display the information log
for message in response_dict['log']:
    if message['level'] >= 20:
        print(message['prefix']+message['message'])

2020-03-03 17:40:31.284320 INFO: ARAXQuery launching
2020-03-03 17:40:31.284340 INFO: Examine input query for needed information for dispatch
2020-03-03 17:40:31.284348 INFO: Found input processing plan. Sending to the ProcessingPlanExecutor
2020-03-03 17:40:31.286784 INFO: Parsing input actions list
2020-03-03 17:40:31.287216 INFO: Adding a QueryNode to Message with parameters {'id': 'n00', 'curie': None, 'name': 'DOID:14330', 'type': None, 'is_set': None}
2020-03-03 17:40:31.287614 INFO: Adding a QueryNode to Message with parameters {'id': 'n01', 'curie': None, 'name': None, 'type': 'protein', 'is_set': 'true'}
2020-03-03 17:40:31.287738 INFO: Adding a QueryNode to Message with parameters {'id': 'n02', 'curie': None, 'name': None, 'type': 'chemical_substance', 'is_set': None}
2020-03-03 17:40:31.287840 INFO: Adding a QueryEdge to Message with parameters {'id': 'e00', 'source_id': 'n00', 'target_id': 'n01', 'type': None}
2020-03-03 17:40:31.287874 INFO: Adding a QueryEdge to Message w

In [61]:
# These URLs provide direct access to resulting data and GUI
print(f"Data: {response_dict['id']}")
if response_dict['id'] is not None:
    match = re.search(r'(\d+)$', response_dict['id'])
    if match:
        print(f"GUI: https://arax.rtx.ai/?m={match.group(1)}")

Data: https://arax.rtx.ai/api/rtx/v1/message/1355
GUI: https://arax.rtx.ai/?m=1355
